Import all important libs

In [ ]:
import numpy as np
import sympy as smp
from sympy import *
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import PillowWriter

Define all symbols using sympy. 

In [ ]:
t, g = smp.symbols('t g')
mc, m1, m2 = smp.symbols('mc m1 m2')
L1, L2 = smp.symbols('L1 L2')
M = mc + m1 + m2

$x_c$ , $\theta_1$ and $\theta_2$ are functions of time and are the important motion variables.

In [ ]:
xc, the1, the2 = smp.symbols(r'x_c \theta_1, \theta_2', cls=smp.Function)

Explicitly write them as functions of time $t$:

In [ ]:
xc = xc(t)
the1 = the1(t)
the2 = the2(t)

check functionality 

In [ ]:
xc

Define derivatives and second derivatives

In [ ]:
xc_d    = smp.diff(xc, t)
xc_dd   = smp.diff(xc_d, t)
the1_d  = smp.diff(the1, t)
the2_d  = smp.diff(the2, t)
the1_dd = smp.diff(the1_d, t)
the2_dd = smp.diff(the2_d, t)

Define $x_1$, $y_1$, $x_2$, and $y_2$ written in terms of the parameters above.

In [ ]:
x1 = xc + L1*smp.sin(the1)
y1 = -L1*smp.cos(the1)
x2 =  x1 +L2*smp.sin(the2)
y2 = y1 -L1*smp.cos(the1)-L2*smp.cos(the2)
F  = M * xc_dd

Then use these to define kinetic and potential energy for each mass. Obtain the Lagrangian

In [ ]:
# Kinetic
Tc = 1/2 * M * xc_d**2
T1 = 1/2 * m1 * (smp.diff(x1, t)**2 + smp.diff(y1, t)**2)
T2 = 1/2 * m2 * (smp.diff(x2, t)**2 + smp.diff(y2, t)**2)
T = T1+T2
# Potential
V1 = m1*g*y1
V2 = m2*g*y2
V = V1 + V2
# Lagrangian
L = T-V



In [ ]:
L

Get Lagrange's equations
$$\frac{\partial L}{\partial x_c}      - \frac{d}{dt}\frac{\partial L}{\partial \dot{x_c}}      = F$$
$$\frac{\partial L}{\partial \theta_1} - \frac{d}{dt}\frac{\partial L}{\partial \dot{\theta_1}} = 0$$
$$\frac{\partial L}{\partial \theta_2} - \frac{d}{dt}\frac{\partial L}{\partial \dot{\theta_2}} = 0$$

In [ ]:
LEC = smp.diff(L, xc).simplify()   - smp.diff(smp.diff(L, xc_d), t).simplify() -F
LE1 = smp.diff(L, the1).simplify() - smp.diff(smp.diff(L, the1_d), t).simplify()
LE2 = smp.diff(L, the2).simplify() - smp.diff(smp.diff(L, the2_d), t).simplify()

Solve Lagranges equations (this assumes that `LEC` ,`LE1` and `LE2` are all equal to zero)

In [ ]:
solc = smp.solve([LEC],(xc_dd), simplify=False, rational=False)

In [ ]:
solc[xc_dd].simplify()

In [ ]:
sol1 = smp.solve([LE1], (the1_dd), simplify=False, rational=False)

In [ ]:
sol2 = smp.solve([LE2], (the2_dd), simplify=False, rational=False)

In [ ]:
print("the1_dd = ")
sol1[the1_dd].simplify()


In [ ]:
print("\n the2_dd = ")
sol2[the2_dd].simplify()

In [ ]:
F = smp.symbols('F')



These are three Equations are second order ODEs! In python we can only solve systems of first order ODEs. Any system of second order ODEs can be converted as follows:

1. First define $z_c = dx_c/dt$ and $dz_c/dt = d^2x_c/dt$ 
2. Then $z_1 = d\theta_1/dt$ and $z_2=d\theta_2/dt$
3. Then $dz_1/dt = d^2\theta_1/dt^2$ and $dz_2/dt = d^2\theta_2/dt^2$


We need to convert the **symbolic** expressions above to numerical functions so we can use them in a numerical python solver. For this we use `smp.lambdify`

In [ ]:
dzcdt_f = lambdify((t, g, mc, m1, m2, L1, L2, xc, the1, the2, xc_d, the1_d, the2_d, the1_dd, the2_dd), solc[xc_dd].simplify())
dz1dt_f = lambdify((t, g, mc, m1, m2, L1, L2, xc, the1, the2, xc_d, the1_d, the2_d, xc_dd, the2_dd), sol1[the1_dd].simplify())
dz2dt_f = lambdify((t, g, mc, m1, m2, L1, L2, xc, the1, the2, xc_d, the1_d, the2_d, xc_dd, the1_dd), sol2[the2_dd].simplify())
dxcdt_f   = smp.lambdify(xc_d, xc_d)
dthe1dt_f = smp.lambdify(the1_d, the1_d)
dthe2dt_f = smp.lambdify(the2_d, the2_d)

Now define $\vec{S} = (x_c, z_c, \theta_1, z_1, \theta_2, z_2)$. IF we're going to use an ODE solver in python, we need to write a function that takes in $\vec{S}$ and $t$ and returns $d\vec{S}/dt$. In other words, we need to define $d\vec{S}/dt (\vec{S}, t)$

* Our system of ODEs can be fully specified using $d\vec{S}/dt$ and depends only on $\vec{S}$ and $t$

In [ ]:
def dSdt(S, t, g, mc, m1, m2, L1, L2):
     the1, z1, the2, z2, xc, zc = S 
     return[
         dxcdt_f(zc),
         dzcdt_f(t,  g, mc, m1, m2, L1, L2, xc, the1, the2, zc, z1, z2,dz1dt_f, dz2dt_f),
         dthe1dt_f(z1),
         dz1dt_f(t,  g, mc, m1, m2, L1, L2, xc, the1, the2, zc, z1, z2,dzcdt_f, dz2dt_f),
         dthe2dt_f(z2),
         dz2dt_f(t,  g, mc, m1, m2, L1, L2, xc, the1, the2, zc, z1, z2,dzcdt_f, dz1dt_f),
     ]

Solve the system of ODEs using scipys `odeint` method

In [ ]:
t = np.linspace(0, 40, 1001)
y0 = [0, 0, 0, 0, 0, 0]
g = 9.81
mc=1
m1=0.4
m2=0.2
L1 = 2
L2 = 1
ans = odeint(dSdt, y0=y0, t=t, args=(g, mc, m1, m2, L1, L2))

25 times per second (number of data points). This will be important for animating later on.

In [ ]:
ans.T

Can obtain $\theta_1(t)$ and $\theta_2(t)$ from the answer

In [ ]:
the1 = ans.T[0]
the2 = ans.T[2]

In [ ]:
plt.plot(t, the2)

Here's a function that takes in $\theta_1$ and $\theta_2$ and returns the location (x,y) of the two masses.

In [ ]:
def get_x1y1x2y2(t, the1, the2, L1, L2):
    return (L1*np.sin(the1),
            -L1*np.cos(the1),
            L1*np.sin(the1) + L2*np.sin(the2),
            -L1*np.cos(the1) - L2*np.cos(the2))

x1, y1, x2, y2 = get_x1y1x2y2(t, ans.T[0], ans.T[2], L1, L2)

Then we can make an animation

In [ ]:
def animate(i):
    ln1.set_data([0, x1[i], x2[i]], [0, y1[i], y2[i]])
    
fig, ax = plt.subplots(1,1, figsize=(8,8))
ax.set_facecolor('k')
ax.get_xaxis().set_ticks([])    # enable this to hide x axis ticks
ax.get_yaxis().set_ticks([])    # enable this to hide y axis ticks
ln1, = plt.plot([], [], 'ro--', lw=3, markersize=8)
ax.set_ylim(-4,4)
ax.set_xlim(-4,4)
ani = animation.FuncAnimation(fig, animate, frames=1000, interval=50)
ani.save('pen.gif',writer='pillow',fps=25)